In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

c:\Users\mlisi\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
f = np.load("moonboard.npz")
X_train, y_train = f['x_train'], f['y_train']
X_test,  y_test  = f['x_test'],  f['y_test']

In [3]:
set(y_train) #grade labels!

{0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [4]:
# There is only one climb labeled with a grade of 0, so we will remove it.
# We need to remove the climb itself as well as the corresponding label.
np.where(y_train == 0)

(array([4099], dtype=int64),)

In [5]:
X_train = np.delete(X_train, 4099, axis=0)
y_train = np.delete(y_train, 4099)

In [6]:
y_test = y_test - 4 # normalizing to 0-12 for sparse categorical cross-entropy
y_train = y_train - 4

In [7]:
X_train.shape

(9523, 18, 11)

In [8]:
y_train.shape

(9523,)

In [14]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(18,11,1)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(13, activation='softmax')

    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 16, 9, 32)         320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 4, 32)         0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 8, 4, 32)         128       
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 2, 64)          18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 3, 1, 64)         0         
 2D)                                                             
                                                      

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
149/149 [==============================] - 1s 4ms/step - loss: 2.0062 - accuracy: 0.2914
Epoch 2/100
149/149 [==============================] - 1s 4ms/step - loss: 1.7939 - accuracy: 0.3486
Epoch 3/100
149/149 [==============================] - 1s 4ms/step - loss: 1.7104 - accuracy: 0.3750
Epoch 4/100
149/149 [==============================] - 1s 4ms/step - loss: 1.6515 - accuracy: 0.3878
Epoch 5/100
149/149 [==============================] - 1s 4ms/step - loss: 1.6151 - accuracy: 0.4032
Epoch 6/100
149/149 [==============================] - 1s 4ms/step - loss: 1.5595 - accuracy: 0.4229
Epoch 7/100
149/149 [==============================] - 1s 4ms/step - loss: 1.5252 - accuracy: 0.4338
Epoch 8/100
149/149 [==============================] - 1s 4ms/step - loss: 1.4839 - accuracy: 0.4492
Epoch 9/100
149/149 [==============================] - 1s 4ms/step - loss: 1.4461 - accuracy: 0.4609
Epoch 10/100
149/149 [==============================] - 1s 4ms/step - loss: 1.4242 - accura

In [16]:
model.evaluate(X_test, y_test)

75/75 [==============================] - 0s 1ms/step - loss: 2.0835 - accuracy: 0.6772


[2.083486795425415, 0.6771620512008667]

In [24]:
nimbus = np.array([
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,1],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

nimbus = nimbus.reshape((1, 18, 11, 1))

easy_prob = np.array([
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

easy_prob = easy_prob.reshape((1,18,11,1))

whispering_aspens = np.array([
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0],
])

whispering_aspens = whispering_aspens.reshape((1,18,11,1))

nimbus_pred = model.predict(nimbus)
ep_output = model.predict(easy_prob)
aspens_pred = model.predict(whispering_aspens)

1/1 [==============================] - 0s 12ms/step


In [14]:
nimbus_pred

array([[8.5877337e-02, 3.7761626e-01, 3.7774161e-01, 7.0042118e-02,
        5.5505749e-02, 9.8595340e-03, 1.7009310e-02, 2.2283439e-03,
        1.4325357e-03, 4.0618697e-04, 2.2029196e-04, 4.2702287e-04,
        1.6337543e-03]], dtype=float32)

In [18]:
np.argmax(ep_output)

0

In [19]:
np.argmax(nimbus_pred)

2

In [26]:
np.argmax(aspens_pred)

7